# Test CSW QUERY

the following will run a query by posting an  XML file via CURL 


At the moment the following Query are supported:
* Group A
    - `anytext`
    - `Time`
    - `BBOX`
    - `anytext` *on field*

* Group B
    - `anytext` **AND** `Time`
    - `anytext` **AND** `BBOX`
    - `anytext` **AND** `Time` **AND** `BBOX`
  
* Group C
    - `anytext` *on field*  **AND** `Time`
    - `anytext` *on field*  **AND** `BBOX`
    - `anytext` *on field* **AND** `Time` **AND** `BBOX`

* Group D
    - `Time` **AND** `BBOX`
    
* Group E
    - `anytext` *on field*  **AND** `BBOX` OR `anytext` *on field*  **AND** `BBOX`

In [1]:
from IPython.core.magic import register_line_cell_magic


@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, "w") as f:
        f.write(cell.format(**globals()))

In [2]:
import xml.dom.minidom


def prettify_xml(xml_string):
    dom = xml.dom.minidom.parseString(xml_string)
    pretty_xml_string = dom.toprettyxml(indent="    ")
    return pretty_xml_string

In [3]:
from IPython.display import HTML, display

# Display the pretty-printed XML in the notebook

In [4]:
csw_test_endpoint = "https://test.csw.met.no"
ElementSetName = "full"

field_to_query = "title"
text_to_query = "ice"

start_time = "2018-09-18 09:00"
end_time = "2020-10-18 09:00"

lowerCorner = "60 0"
upperCorner = "90 180"

In [5]:
from geolinks import sniff_link
from owslib.csw import CatalogueServiceWeb
from owslib.fes import (
    And,
    BBox,
    Or,
    PropertyIsEqualTo,
    PropertyIsGreaterThanOrEqualTo,
    PropertyIsLessThanOrEqualTo,
    PropertyIsLike,
)

csw = CatalogueServiceWeb(csw_test_endpoint, timeout=60)

## Group A

### `anytext`

* Corresponding SOLR JSON Query:

```json
{
    "q": "full_text:(ice)",
    "q.op": "OR",
    "start": 0,
    "rows": "5",
    "fq": [
        "metadata_status:Active",
        "collection:(ADC)"
    ]
}
```

#### CSW XML

In [6]:
%%writetemplate test_xml_query_anytext.xml
<?xml version="1.0" encoding="ISO-8859-1" standalone="no"?>
<csw:GetRecords xmlns:apiso="http://www.opengis.net/cat/csw/apiso/1.0" xmlns:csw="http://www.opengis.net/cat/csw/2.0.2" xmlns:ogc="http://www.opengis.net/ogc" service="CSW" version="2.0.2" resultType="results" startPosition="1" maxRecords="5" outputFormat="application/xml" outputSchema="http://www.isotc211.org/2005/gmd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.opengis.net/cat/csw/2.0.2 http://schemas.opengis.net/csw/2.0.2/CSW-discovery.xsd" xmlns:gml="http://www.opengis.net/gml" xmlns:gmd="http://www.isotc211.org/2005/gmd">
    <csw:Query typeNames="gmd:MD_Metadata">
        <csw:ElementSetName>{ElementSetName}</csw:ElementSetName>
        <csw:Constraint version="1.1.0">
            <ogc:Filter>
                    <ogc:PropertyIsLike wildCard="*" singleChar="?" escapeChar="\\" matchCase="false">
                        <ogc:PropertyName>apiso:AnyText</ogc:PropertyName>
                        <ogc:Literal>{text_to_query}</ogc:Literal>
                    </ogc:PropertyIsLike>
            </ogc:Filter>
        </csw:Constraint>
    </csw:Query>
</csw:GetRecords>

In [7]:
!curl -s -H "Content-Type: text/xml" -X POST -d @test_xml_query_anytext.xml {csw_test_endpoint} | grep -oP 'numberOfRecordsMatched="\K[^"]+'

135225


#### Python

In [8]:
# Create a filter
filter = PropertyIsLike(
    propertyname="apiso:AnyText",
    literal=f"{text_to_query}",
    wildCard="*",
    singleChar="?",
    escapeChar="\\",
)

# Use the filter in a CSW getrecords request
records = csw.getrecords2(
    constraints=[filter],
    startposition=1,
    maxrecords=5,
)

In [9]:
csw.results['matches']

135225

### `Time`


* Corresponding SOLR JSON Query:

```json
{
    "q": "*:*",
    "q.op": "OR",
    "start": 0,
    "rows": "5",
    "fq": [
        "metadata_status:Active",
        "temporal_extent_start_date:[2018-09-18T09:00:00Z TO *]",
        "temporal_extent_end_date:[* TO 2020-10-18T09:00:00Z]",
        "collection:(ADC)"
    ]
}
```

#### CSW XML

In [10]:
%%writetemplate test_xml_query_time.xml
<?xml version="1.0" encoding="ISO-8859-1" standalone="no"?>
<csw:GetRecords xmlns:apiso="http://www.opengis.net/cat/csw/apiso/1.0" xmlns:csw="http://www.opengis.net/cat/csw/2.0.2" xmlns:ogc="http://www.opengis.net/ogc" service="CSW" version="2.0.2" resultType="results" startPosition="1" maxRecords="5" outputFormat="application/xml" outputSchema="http://www.isotc211.org/2005/gmd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.opengis.net/cat/csw/2.0.2 http://schemas.opengis.net/csw/2.0.2/CSW-discovery.xsd" xmlns:gml="http://www.opengis.net/gml" xmlns:gmd="http://www.isotc211.org/2005/gmd">
    <csw:Query typeNames="gmd:MD_Metadata">
        <csw:ElementSetName>{ElementSetName}</csw:ElementSetName>
        <csw:Constraint version="1.1.0">
     <ogc:Filter>
       <ogc:And>
         <ogc:PropertyIsLessThanOrEqualTo>
           <ogc:PropertyName>apiso:TempExtent_begin</ogc:PropertyName>
           <ogc:Literal>{end_time}</ogc:Literal>
         </ogc:PropertyIsLessThanOrEqualTo>
        <ogc:PropertyIsGreaterThanOrEqualTo>
           <ogc:PropertyName>apiso:TempExtent_end</ogc:PropertyName>
           <ogc:Literal>{start_time}</ogc:Literal>
         </ogc:PropertyIsGreaterThanOrEqualTo>
       </ogc:And>
     </ogc:Filter>
   </csw:Constraint>
    </csw:Query>
</csw:GetRecords>

In [11]:
!curl -s -H "Content-Type: text/xml" -X POST -d @test_xml_query_time.xml {csw_test_endpoint} | grep -oP 'numberOfRecordsMatched="\K[^"]+'

39075


#### Python

In [12]:
# Create filters
filter1 = PropertyIsLessThanOrEqualTo(
    propertyname="apiso:TempExtent_begin", literal=f"{end_time}"
)
filter2 = PropertyIsGreaterThanOrEqualTo(
    propertyname="apiso:TempExtent_end", literal=f"{start_time}"
)

# Combine filters
filter = And([filter1, filter2])

# Use the filter in a CSW getrecords request
csw.getrecords2(
    constraints=[filter],
    esn=f"{ElementSetName}",
    startposition=1,
    maxrecords=5,
)

In [13]:
csw.results['matches']

39075

### `BBOX`



* Corresponding SOLR JSON Query:

```json
{
    "q": "*:*",
    "q.op": "OR",
    "start": 0,
    "rows": "5",
    "fq": [
        "metadata_status:Active",
        "{!field f=bbox score=overlapRatio}Within(ENVELOPE(60.0,90.0,180.0,0.0))",
        "collection:(ADC)"
    ]
}
```

#### CSW XML

In [14]:
%%writetemplate test_xml_query_bbox.xml
<?xml version="1.0" encoding="ISO-8859-1" standalone="no"?>
<csw:GetRecords xmlns:apiso="http://www.opengis.net/cat/csw/apiso/1.0" xmlns:csw="http://www.opengis.net/cat/csw/2.0.2" xmlns:ogc="http://www.opengis.net/ogc" service="CSW" version="2.0.2" resultType="results" startPosition="1" maxRecords="5" outputFormat="application/xml" outputSchema="http://www.isotc211.org/2005/gmd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.opengis.net/cat/csw/2.0.2 http://schemas.opengis.net/csw/2.0.2/CSW-discovery.xsd" xmlns:gml="http://www.opengis.net/gml" xmlns:gmd="http://www.isotc211.org/2005/gmd">
    <csw:Query typeNames="gmd:MD_Metadata">
        <csw:ElementSetName>{ElementSetName}</csw:ElementSetName>
        <csw:Constraint version="1.1.0">
            <ogc:Filter>
                <ogc:BBOX>
                    <ogc:PropertyName>apiso:BoundingBox</ogc:PropertyName>
                    <gml:Envelope>
                        <gml:lowerCorner>{lowerCorner}</gml:lowerCorner>
                        <gml:upperCorner>{upperCorner}</gml:upperCorner>
                    </gml:Envelope>
                </ogc:BBOX>
            </ogc:Filter>
        </csw:Constraint>
    </csw:Query>
</csw:GetRecords>

In [15]:
!curl -s -H "Content-Type: text/xml" -X POST -d @test_xml_query_bbox.xml {csw_test_endpoint} | grep -oP 'numberOfRecordsMatched="\K[^"]+'

6449


#### Python

In [16]:
# Create a filter
# Define your bounding box coordinates
bbox = lowerCorner.split() + upperCorner.split()

# Create a filter
bbox_filter = BBox(bbox)
# Use the filter in a CSW getrecords request
csw.getrecords2(
    constraints=[bbox_filter],
    esn=f"{ElementSetName}",
    startposition=1,
    maxrecords=5,
)

In [17]:
csw.results['matches']

6449

### `anytext` *on field*



* Corresponding SOLR JSON Query:

```json
{
    "q": "*:*",
    "q.op": "OR",
    "start": 0,
    "rows": "5",
    "fq": [
        "metadata_status:Active",
        "title:(ice)",
        "collection:(ADC)"
    ]
}
```

#### CSW XML

In [19]:
%%writetemplate test_xml_query_anytext_on_field.xml
<?xml version="1.0" encoding="ISO-8859-1" standalone="no"?>
<csw:GetRecords xmlns:apiso="http://www.opengis.net/cat/csw/apiso/1.0" xmlns:csw="http://www.opengis.net/cat/csw/2.0.2" xmlns:ogc="http://www.opengis.net/ogc" service="CSW" version="2.0.2" resultType="results" startPosition="1" maxRecords="5" outputFormat="application/xml" outputSchema="http://www.isotc211.org/2005/gmd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.opengis.net/cat/csw/2.0.2 http://schemas.opengis.net/csw/2.0.2/CSW-discovery.xsd" xmlns:gml="http://www.opengis.net/gml" xmlns:gmd="http://www.isotc211.org/2005/gmd">
    <csw:Query typeNames="gmd:MD_Metadata">
        <csw:ElementSetName>{ElementSetName}</csw:ElementSetName>
        <csw:Constraint version="1.1.0">
            <ogc:Filter>
                    <ogc:PropertyIsLike wildCard="*" singleChar="?" escapeChar="\\" matchCase="false">
                        <ogc:PropertyName>dc:{field_to_query}</ogc:PropertyName>
                        <ogc:Literal>{text_to_query}</ogc:Literal>
                    </ogc:PropertyIsLike>
            </ogc:Filter>
        </csw:Constraint>
    </csw:Query>
</csw:GetRecords>

In [20]:
!curl -s -H "Content-Type: text/xml" -X POST -d @test_xml_query_anytext_on_field.xml {csw_test_endpoint} | grep -oP 'numberOfRecordsMatched="\K[^"]+'

58611


#### Python

In [21]:
filter = PropertyIsLike(
    propertyname=f"dc:{field_to_query}",
    literal=f"{text_to_query}",
    wildCard="*",
    singleChar="?",
    escapeChar="\\",
)

csw.getrecords2(
    constraints=[filter],
    esn=f"{ElementSetName}",
    startposition=1,
    maxrecords=5,
)

In [22]:
csw.results['matches']

58611

## Group B

### `anytext` **AND** `Time`

* Corresponding SOLR JSON Query:

```json
{
    "q": "full_text:(ice)",
    "q.op": "OR",
    "start": 0,
    "rows": "5",
    "fq": [
        "metadata_status:Active",
        "temporal_extent_start_date:[2018-09-18T09:00:00Z TO *]",
        "temporal_extent_end_date:[* TO 2020-10-18T09:00:00Z]",
        "collection:(ADC)"
    ]
}
```

#### CSW XML

In [24]:
%%writetemplate test_xml_query_anytext_AND_time.xml
<?xml version="1.0" encoding="ISO-8859-1" standalone="no"?>
<csw:GetRecords xmlns:apiso="http://www.opengis.net/cat/csw/apiso/1.0" xmlns:csw="http://www.opengis.net/cat/csw/2.0.2" xmlns:ogc="http://www.opengis.net/ogc" service="CSW" version="2.0.2" resultType="results" startPosition="1" maxRecords="5" outputFormat="application/xml" outputSchema="http://www.isotc211.org/2005/gmd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.opengis.net/cat/csw/2.0.2 http://schemas.opengis.net/csw/2.0.2/CSW-discovery.xsd" xmlns:gml="http://www.opengis.net/gml" xmlns:gmd="http://www.isotc211.org/2005/gmd">
    <csw:Query typeNames="gmd:MD_Metadata">
        <csw:ElementSetName>{ElementSetName}</csw:ElementSetName>
        <csw:Constraint version="1.1.0">
            <ogc:Filter>
                <ogc:And>
                    <ogc:PropertyIsLike wildCard="*" singleChar="?" escapeChar="\\" matchCase="false">
                        <ogc:PropertyName>apiso:AnyText</ogc:PropertyName>
                        <ogc:Literal>{text_to_query}</ogc:Literal>
                    </ogc:PropertyIsLike>
                    <ogc:PropertyIsLessThanOrEqualTo>
                        <ogc:PropertyName>apiso:TempExtent_begin</ogc:PropertyName>
                        <ogc:Literal>{end_time}</ogc:Literal>
                    </ogc:PropertyIsLessThanOrEqualTo>
                    <ogc:PropertyIsGreaterThanOrEqualTo>
                        <ogc:PropertyName>apiso:TempExtent_end</ogc:PropertyName>
                        <ogc:Literal>{start_time}</ogc:Literal>
                    </ogc:PropertyIsGreaterThanOrEqualTo>
                </ogc:And>
            </ogc:Filter>
        </csw:Constraint>
    </csw:Query>
</csw:GetRecords>

In [25]:
!curl -s -H "Content-Type: text/xml" -X POST -d @test_xml_query_anytext_AND_time.xml {csw_test_endpoint} | grep -oP 'numberOfRecordsMatched="\K[^"]+'

2835


#### Python

In [26]:
# Create filters
text_filter = PropertyIsLike(
    propertyname="apiso:AnyText",
    literal=f"{text_to_query}",
    wildCard="*",
    singleChar="?",
    escapeChar="\\",
)
time_filter1 = PropertyIsLessThanOrEqualTo(
    propertyname="apiso:TempExtent_begin", literal=f"{end_time}"
)
time_filter2 = PropertyIsGreaterThanOrEqualTo(
    propertyname="apiso:TempExtent_end", literal=f"{start_time}"
)

# Combine filters
filter = And([text_filter, time_filter1, time_filter2])

# Use the filter in a CSW getrecords request
csw.getrecords2(
    constraints=[filter], esn=f"{ElementSetName}", startposition=1, maxrecords=5
)

In [27]:
csw.results['matches']

2835

### `anytext` **AND** `BBOX`


* Corresponding SOLR JSON Query:

```json
{
    "q": "full_text:(ice)",
    "q.op": "OR",
    "start": 0,
    "rows": "5",
    "fq": [
        "metadata_status:Active",
        "{!field f=bbox score=overlapRatio}Within(ENVELOPE(60.0,90.0,180.0,0.0))",
        "collection:(ADC)"
    ]
}
```

#### CSW XML

In [29]:
%%writetemplate test_xml_query_anytext_AND_BBOX.xml
<?xml version="1.0" encoding="ISO-8859-1" standalone="no"?>
<csw:GetRecords xmlns:apiso="http://www.opengis.net/cat/csw/apiso/1.0" xmlns:csw="http://www.opengis.net/cat/csw/2.0.2" xmlns:ogc="http://www.opengis.net/ogc" service="CSW" version="2.0.2" resultType="results" startPosition="1" maxRecords="5" outputFormat="application/xml" outputSchema="http://www.isotc211.org/2005/gmd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.opengis.net/cat/csw/2.0.2 http://schemas.opengis.net/csw/2.0.2/CSW-discovery.xsd" xmlns:gml="http://www.opengis.net/gml" xmlns:gmd="http://www.isotc211.org/2005/gmd">
    <csw:Query typeNames="gmd:MD_Metadata">
        <csw:ElementSetName>{ElementSetName}</csw:ElementSetName>
        <csw:Constraint version="1.1.0">
            <ogc:Filter>
                <ogc:And>
                    <ogc:PropertyIsLike wildCard="*" singleChar="?" escapeChar="\\" matchCase="false">
                        <ogc:PropertyName>apiso:AnyText</ogc:PropertyName>
                        <ogc:Literal>{text_to_query}</ogc:Literal>
                    </ogc:PropertyIsLike>
                    <ogc:BBOX>
                        <ogc:PropertyName>apiso:BoundingBox</ogc:PropertyName>
                            <gml:Envelope>
                                <gml:lowerCorner>{lowerCorner}</gml:lowerCorner>
                                <gml:upperCorner>{upperCorner}</gml:upperCorner>
                            </gml:Envelope>
                    </ogc:BBOX>
                </ogc:And>
            </ogc:Filter>
        </csw:Constraint>
    </csw:Query>
</csw:GetRecords>

In [30]:
!curl -s -H "Content-Type: text/xml" -X POST -d @test_xml_query_anytext_AND_BBOX.xml {csw_test_endpoint} | grep -oP 'numberOfRecordsMatched="\K[^"]+'

6365


#### Python

In [31]:
# Create filters
text_filter = PropertyIsLike(
    propertyname="apiso:AnyText",
    literal=f"{text_to_query}",
    wildCard="*",
    singleChar="?",
    escapeChar="\\",
)

bbox = lowerCorner.split() + upperCorner.split()

bbox_filter = BBox(bbox)

# Combine filters
filter = And([text_filter, bbox_filter])

# Use the filter in a CSW getrecords request
csw.getrecords2(
    constraints=[filter],
    esn=f"{ElementSetName}",
    startposition=1,
    maxrecords=5,
)

In [32]:
csw.results['matches']

6365

### `anytext` **AND** `Time` **AND** `BBOX`

* Corresponding SOLR JSON Query:

```json
{
    "q": "full_text:(ice)",
    "q.op": "OR",
    "start": 0,
    "rows": "5",
    "fq": [
        "metadata_status:Active",
        "{!field f=bbox score=overlapRatio}Within(ENVELOPE(60.0,90.0,180.0,0.0))",
        "temporal_extent_start_date:[2018-09-18T09:00:00Z TO *]",
        "temporal_extent_end_date:[* TO 2020-10-18T09:00:00Z]",
        "collection:(ADC)"
    ]
}
```

#### CSW XML

In [34]:
%%writetemplate test_xml_query_anytext_AND_time_AND_BBOX.xml
<?xml version="1.0" encoding="ISO-8859-1" standalone="no"?>
<csw:GetRecords xmlns:apiso="http://www.opengis.net/cat/csw/apiso/1.0" xmlns:csw="http://www.opengis.net/cat/csw/2.0.2" xmlns:ogc="http://www.opengis.net/ogc" service="CSW" version="2.0.2" resultType="results" startPosition="1" maxRecords="5" outputFormat="application/xml" outputSchema="http://www.isotc211.org/2005/gmd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.opengis.net/cat/csw/2.0.2 http://schemas.opengis.net/csw/2.0.2/CSW-discovery.xsd" xmlns:gml="http://www.opengis.net/gml" xmlns:gmd="http://www.isotc211.org/2005/gmd">
    <csw:Query typeNames="gmd:MD_Metadata">
        <csw:ElementSetName>{ElementSetName}</csw:ElementSetName>
        <csw:Constraint version="1.1.0">
            <ogc:Filter>
                <ogc:And>
                    <ogc:PropertyIsLike wildCard="*" singleChar="?" escapeChar="\\" matchCase="false">
                        <ogc:PropertyName>apiso:AnyText</ogc:PropertyName>
                        <ogc:Literal>{text_to_query}</ogc:Literal>
                    </ogc:PropertyIsLike>
                    <ogc:BBOX>
                        <ogc:PropertyName>apiso:BoundingBox</ogc:PropertyName>
                        <gml:Envelope>
                            <gml:lowerCorner>{lowerCorner}</gml:lowerCorner>
                            <gml:upperCorner>{upperCorner}</gml:upperCorner>
                        </gml:Envelope>
                    </ogc:BBOX>
                    <ogc:PropertyIsLessThanOrEqualTo>
                        <ogc:PropertyName>apiso:TempExtent_begin</ogc:PropertyName>
                        <ogc:Literal>{end_time}</ogc:Literal>
                    </ogc:PropertyIsLessThanOrEqualTo>
                    <ogc:PropertyIsGreaterThanOrEqualTo>
                        <ogc:PropertyName>apiso:TempExtent_end</ogc:PropertyName>
                        <ogc:Literal>{start_time}</ogc:Literal>
                    </ogc:PropertyIsGreaterThanOrEqualTo>
                </ogc:And>
            </ogc:Filter>
        </csw:Constraint>
    </csw:Query>
</csw:GetRecords>

In [35]:
!curl -s -H "Content-Type: text/xml" -X POST -d @test_xml_query_anytext_AND_time_AND_BBOX.xml {csw_test_endpoint} | grep -oP 'numberOfRecordsMatched="\K[^"]+'

3


#### Python

In [36]:
# Create filters
text_filter = PropertyIsLike(
    propertyname="apiso:AnyText",
    literal=f"{text_to_query}",
    wildCard="*",
    singleChar="?",
    escapeChar="\\",
)
bbox_filter = BBox(lowerCorner.split() + upperCorner.split())
time_filter1 = PropertyIsLessThanOrEqualTo(
    propertyname="apiso:TempExtent_begin", literal=f"{end_time}"
)
time_filter2 = PropertyIsGreaterThanOrEqualTo(
    propertyname="apiso:TempExtent_end", literal=f"{start_time}"
)

# Combine filters
filter = And([text_filter, bbox_filter, time_filter1, time_filter2])

# Use the filter in a CSW getrecords request
csw.getrecords2(
    constraints=[filter],
    esn=f"{ElementSetName}",
    startposition=1,
    maxrecords=5,
)

In [37]:
csw.results['matches']

3

## Group C

### `anytext` *on field*  **AND** `Time`

* Corresponding SOLR JSON Query:

```json
{
    "q": "*:*",
    "q.op": "OR",
    "start": 0,
    "rows": "5",
    "fq": [
        "metadata_status:Active",
        "title:(ice)",
        "temporal_extent_start_date:[2018-09-18T09:00:00Z TO *]",
        "temporal_extent_end_date:[* TO 2020-10-18T09:00:00Z]",
        "collection:(ADC)"
    ]
}
```

#### CSW XML

In [38]:
%%writetemplate test_xml_query_anytext_on_field_AND_time.xml
<?xml version="1.0" encoding="ISO-8859-1" standalone="no"?>
<csw:GetRecords xmlns:apiso="http://www.opengis.net/cat/csw/apiso/1.0" xmlns:csw="http://www.opengis.net/cat/csw/2.0.2" xmlns:ogc="http://www.opengis.net/ogc" service="CSW" version="2.0.2" resultType="results" startPosition="1" maxRecords="5" outputFormat="application/xml" outputSchema="http://www.isotc211.org/2005/gmd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.opengis.net/cat/csw/2.0.2 http://schemas.opengis.net/csw/2.0.2/CSW-discovery.xsd" xmlns:gml="http://www.opengis.net/gml" xmlns:gmd="http://www.isotc211.org/2005/gmd">
    <csw:Query typeNames="gmd:MD_Metadata">
        <csw:ElementSetName>{ElementSetName}</csw:ElementSetName>
        <csw:Constraint version="1.1.0">
            <ogc:Filter>
                <ogc:And>
                    <ogc:PropertyIsLike wildCard="*" singleChar="?" escapeChar="\\" matchCase="false">
                        <ogc:PropertyName>dc:{field_to_query}</ogc:PropertyName>
                        <ogc:Literal>{text_to_query}</ogc:Literal>
                    </ogc:PropertyIsLike>
                    <ogc:PropertyIsLessThanOrEqualTo>
                        <ogc:PropertyName>apiso:TempExtent_begin</ogc:PropertyName>
                        <ogc:Literal>{end_time}</ogc:Literal>
                    </ogc:PropertyIsLessThanOrEqualTo>
                    <ogc:PropertyIsGreaterThanOrEqualTo>
                        <ogc:PropertyName>apiso:TempExtent_end</ogc:PropertyName>
                        <ogc:Literal>{start_time}</ogc:Literal>
                    </ogc:PropertyIsGreaterThanOrEqualTo>
                </ogc:And>
            </ogc:Filter>
        </csw:Constraint>
    </csw:Query>
</csw:GetRecords>

In [39]:
!curl -s -H "Content-Type: text/xml" -X POST -d @test_xml_query_anytext_on_field_AND_time.xml {csw_test_endpoint}  | grep -oP 'numberOfRecordsMatched="\K[^"]+'

2383


#### Python

In [40]:
# Create filters
text_filter = PropertyIsLike(
    propertyname=f"dc:{field_to_query}",
    literal=f"{text_to_query}",
    wildCard="*",
    singleChar="?",
    escapeChar="\\",
)
time_filter1 = PropertyIsLessThanOrEqualTo(
    propertyname="apiso:TempExtent_begin", literal=f"{end_time}"
)
time_filter2 = PropertyIsGreaterThanOrEqualTo(
    propertyname="apiso:TempExtent_end", literal=f"{start_time}"
)

# Combine filters
filter = And([text_filter, time_filter1, time_filter2])

# Use the filter in a CSW getrecords request
csw.getrecords2(
    constraints=[filter],
    esn=f"{ElementSetName}",
    startposition=1,
    maxrecords=5,
)

In [41]:
csw.results['matches']

2383

### `anytext` *on field*  **AND** `BBOX`

* Corresponding SOLR JSON Query:

```json
{
    "q": "*:*",
    "q.op": "OR",
    "start": 0,
    "rows": "5",
    "fq": [
        "metadata_status:Active",
        "title:(ice)",
        "{!field f=bbox score=overlapRatio}Within(ENVELOPE(60.0,90.0,180.0,0.0))",
        "collection:(ADC)"
    ]
}
```

#### CSW XML

In [42]:
%%writetemplate test_xml_query_anytext_on_field_AND_BBOX.xml
<?xml version="1.0" encoding="ISO-8859-1" standalone="no"?>
<csw:GetRecords xmlns:apiso="http://www.opengis.net/cat/csw/apiso/1.0" xmlns:csw="http://www.opengis.net/cat/csw/2.0.2" xmlns:ogc="http://www.opengis.net/ogc" service="CSW" version="2.0.2" resultType="results" startPosition="1" maxRecords="5" outputFormat="application/xml" outputSchema="http://www.isotc211.org/2005/gmd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.opengis.net/cat/csw/2.0.2 http://schemas.opengis.net/csw/2.0.2/CSW-discovery.xsd" xmlns:gml="http://www.opengis.net/gml" xmlns:gmd="http://www.isotc211.org/2005/gmd">
    <csw:Query typeNames="gmd:MD_Metadata">
        <csw:ElementSetName>{ElementSetName}</csw:ElementSetName>
        <csw:Constraint version="1.1.0">
            <ogc:Filter>
                <ogc:And>
                    <ogc:PropertyIsLike wildCard="*" singleChar="?" escapeChar="\\" matchCase="false">
                        <ogc:PropertyName>dc:{field_to_query}</ogc:PropertyName>
                        <ogc:Literal>{text_to_query}</ogc:Literal>
                    </ogc:PropertyIsLike>
                    <ogc:BBOX>
                        <ogc:PropertyName>apiso:BoundingBox</ogc:PropertyName>
                            <gml:Envelope>
                                <gml:lowerCorner>{lowerCorner}</gml:lowerCorner>
                                <gml:upperCorner>{upperCorner}</gml:upperCorner>
                            </gml:Envelope>
                    </ogc:BBOX>
                </ogc:And>
            </ogc:Filter>
        </csw:Constraint>
    </csw:Query>
</csw:GetRecords>

In [43]:
!curl -s -H "Content-Type: text/xml" -X POST -d @test_xml_query_anytext_on_field_AND_BBOX.xml {csw_test_endpoint} | grep -oP 'numberOfRecordsMatched="\K[^"]+'

0


#### Python

In [44]:
# Create filters
text_filter = PropertyIsLike(
    propertyname=f"dc:{field_to_query}",
    literal=f"{text_to_query}",
    wildCard="*",
    singleChar="?",
    escapeChar="\\",
)
bbox_filter = BBox(lowerCorner.split() + upperCorner.split())

# Combine filters
filter = And([text_filter, bbox_filter])

# Use the filter in a CSW getrecords request
csw.getrecords2(
    constraints=[filter],
    esn=f"{ElementSetName}",
    startposition=1,
    maxrecords=5,
)

In [45]:
csw.results['matches']

0

### `anytext` *on field* **AND** `Time` **AND** `BBOX`

* Corresponding SOLR JSON Query:

```json
{
    "q": "*:*",
    "q.op": "OR",
    "start": 0,
    "rows": "5",
    "fq": [
        "metadata_status:Active",
        "title:(ice)",
        "{!field f=bbox score=overlapRatio}Within(ENVELOPE(60.0,90.0,180.0,0.0))",
        "temporal_extent_start_date:[2018-09-18T09:00:00Z TO *]",
        "temporal_extent_end_date:[* TO 2020-10-18T09:00:00Z]",
        "collection:(ADC)"
    ]
}
```

#### CSW XML

In [47]:
%%writetemplate test_xml_query_anytext_on_field_AND_Time_AND_BBOX.xml
<?xml version="1.0" encoding="ISO-8859-1" standalone="no"?>
<csw:GetRecords xmlns:apiso="http://www.opengis.net/cat/csw/apiso/1.0" xmlns:csw="http://www.opengis.net/cat/csw/2.0.2" xmlns:ogc="http://www.opengis.net/ogc" service="CSW" version="2.0.2" resultType="results" startPosition="1" maxRecords="5" outputFormat="application/xml" outputSchema="http://www.isotc211.org/2005/gmd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.opengis.net/cat/csw/2.0.2 http://schemas.opengis.net/csw/2.0.2/CSW-discovery.xsd" xmlns:gml="http://www.opengis.net/gml" xmlns:gmd="http://www.isotc211.org/2005/gmd">
    <csw:Query typeNames="gmd:MD_Metadata">
        <csw:ElementSetName>{ElementSetName}</csw:ElementSetName>
        <csw:Constraint version="1.1.0">
            <ogc:Filter>
                <ogc:And>
                    <ogc:PropertyIsLike wildCard="*" singleChar="?" escapeChar="\\" matchCase="false">
                        <ogc:PropertyName>dc:{field_to_query}</ogc:PropertyName>
                        <ogc:Literal>{text_to_query}</ogc:Literal>
                    </ogc:PropertyIsLike>
                    <ogc:BBOX>
                        <ogc:PropertyName>apiso:BoundingBox</ogc:PropertyName>
                            <gml:Envelope>
                                <gml:lowerCorner>{lowerCorner}</gml:lowerCorner>
                                <gml:upperCorner>{upperCorner}</gml:upperCorner>
                            </gml:Envelope>
                    </ogc:BBOX>
                    <ogc:PropertyIsLessThanOrEqualTo>
                        <ogc:PropertyName>apiso:TempExtent_begin</ogc:PropertyName>
                        <ogc:Literal>{end_time}</ogc:Literal>
                    </ogc:PropertyIsLessThanOrEqualTo>
                    <ogc:PropertyIsGreaterThanOrEqualTo>
                        <ogc:PropertyName>apiso:TempExtent_end</ogc:PropertyName>
                        <ogc:Literal>{start_time}</ogc:Literal>
                    </ogc:PropertyIsGreaterThanOrEqualTo>
                </ogc:And>
            </ogc:Filter>
        </csw:Constraint>
    </csw:Query>
</csw:GetRecords>

In [48]:
!curl -s -H "Content-Type: text/xml" -X POST -d @test_xml_query_anytext_on_field_AND_Time_AND_BBOX.xml {csw_test_endpoint} | grep -oP 'numberOfRecordsMatched="\K[^"]+'

0


#### Python

In [49]:
# Create filters
text_filter = PropertyIsLike(
    propertyname=f"dc:{field_to_query}",
    literal=f"{text_to_query}",
    wildCard="*",
    singleChar="?",
    escapeChar="\\",
)
bbox_filter = BBox(lowerCorner.split() + upperCorner.split())
time_filter1 = PropertyIsLessThanOrEqualTo(
    propertyname="apiso:TempExtent_begin", literal=f"{end_time}"
)
time_filter2 = PropertyIsGreaterThanOrEqualTo(
    propertyname="apiso:TempExtent_end", literal=f"{start_time}"
)

# Combine filters
filter = And([text_filter, bbox_filter, time_filter1, time_filter2])

# Use the filter in a CSW getrecords request
csw.getrecords2(
    constraints=[filter],
    esn=f"{ElementSetName}",
    startposition=1,
    maxrecords=5,
)

In [50]:
csw.results['matches']

0

## Group D

### `Time` **AND** `BBOX`

* Corresponding SOLR JSON Query:

```json
{
    "q": "*:*",
    "q.op": "OR",
    "start": 0,
    "rows": "5",
    "fq": [
        "metadata_status:Active",
        "{!field f=bbox score=overlapRatio}Within(ENVELOPE(60.0,90.0,180.0,0.0))",
        "temporal_extent_start_date:[2018-09-18T09:00:00Z TO *]",
        "temporal_extent_end_date:[* TO 2020-10-18T09:00:00Z]",
        "collection:(ADC)"
    ]
}
```

#### CSW XML

In [51]:
%%writetemplate test_xml_query_Time_AND_BBOX.xml
<?xml version="1.0" encoding="ISO-8859-1" standalone="no"?>
<csw:GetRecords xmlns:apiso="http://www.opengis.net/cat/csw/apiso/1.0" xmlns:csw="http://www.opengis.net/cat/csw/2.0.2" xmlns:ogc="http://www.opengis.net/ogc" service="CSW" version="2.0.2" resultType="results" startPosition="1" maxRecords="5" outputFormat="application/xml" outputSchema="http://www.isotc211.org/2005/gmd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.opengis.net/cat/csw/2.0.2 http://schemas.opengis.net/csw/2.0.2/CSW-discovery.xsd" xmlns:gml="http://www.opengis.net/gml" xmlns:gmd="http://www.isotc211.org/2005/gmd">
    <csw:Query typeNames="gmd:MD_Metadata">
        <csw:ElementSetName>{ElementSetName}</csw:ElementSetName>
        <csw:Constraint version="1.1.0">
            <ogc:Filter>
                <ogc:And>
                    <ogc:BBOX>
                        <ogc:PropertyName>apiso:BoundingBox</ogc:PropertyName>
                            <gml:Envelope>
                                <gml:lowerCorner>{lowerCorner}</gml:lowerCorner>
                                <gml:upperCorner>{upperCorner}</gml:upperCorner>
                            </gml:Envelope>
                    </ogc:BBOX>
                    <ogc:PropertyIsLessThanOrEqualTo>
                        <ogc:PropertyName>apiso:TempExtent_begin</ogc:PropertyName>
                        <ogc:Literal>{end_time}</ogc:Literal>
                    </ogc:PropertyIsLessThanOrEqualTo>
                    <ogc:PropertyIsGreaterThanOrEqualTo>
                        <ogc:PropertyName>apiso:TempExtent_end</ogc:PropertyName>
                        <ogc:Literal>{start_time}</ogc:Literal>
                    </ogc:PropertyIsGreaterThanOrEqualTo>
                </ogc:And>
            </ogc:Filter>
        </csw:Constraint>
    </csw:Query>
</csw:GetRecords>

In [52]:
!curl -s -H "Content-Type: text/xml" -X POST -d @test_xml_query_Time_AND_BBOX.xml {csw_test_endpoint} | grep -oP 'numberOfRecordsMatched="\K[^"]+'

34


#### Python

In [53]:
# Create filters
bbox_filter = BBox(lowerCorner.split() + upperCorner.split())
time_filter1 = PropertyIsLessThanOrEqualTo(
    propertyname="apiso:TempExtent_begin", literal=f"{end_time}"
)
time_filter2 = PropertyIsGreaterThanOrEqualTo(
    propertyname="apiso:TempExtent_end", literal=f"{start_time}"
)

# Combine filters
filter = And([bbox_filter, time_filter1, time_filter2])

# Use the filter in a CSW getrecords request
csw.getrecords2(
    constraints=[filter],
    esn=f"{ElementSetName}",
    startposition=1,
    maxrecords=5,
)

In [54]:
csw.results['matches']

34

## Group E

### OR Query 
OR query inside AND have (*yet*) some limitation, however, the case is identified in the code
It could be useful to identify multiple `AnyText on field` and `BBOX` query
At the moment if a multiple `TIME` is not supported, and the provided the code will silently IGNORE such constraint

* Corresponding SOLR JSON Query:

```json
{
    "q": "(title:temperature && _query_:\"{!field f=bbox score=overlapRatio}Within(ENVELOPE(60.0,90.0,180.0,0.0))\") OR (title:area && _query_:\"{!field f=bbox score=overlapRatio}Within(ENVELOPE(60.0,90.0,180.0,0.0))\")",
    "q.op": "OR",
    "start": 0,
    "rows": "5",
    "fq": [
        "metadata_status:Active",
        "collection:(ADC)"
    ]
}
```

#### CSW XML

In [55]:
%%writetemplate test_xml_query_anytext_on_field_AND_BBOX_OR_testing_3.xml
<?xml version="1.0" encoding="ISO-8859-1" standalone="no"?>
<csw:GetRecords xmlns:apiso="http://www.opengis.net/cat/csw/apiso/1.0" xmlns:csw="http://www.opengis.net/cat/csw/2.0.2" xmlns:ogc="http://www.opengis.net/ogc" service="CSW" version="2.0.2" resultType="results" startPosition="1" maxRecords="5" outputFormat="application/xml" outputSchema="http://www.isotc211.org/2005/gmd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.opengis.net/cat/csw/2.0.2 http://schemas.opengis.net/csw/2.0.2/CSW-discovery.xsd" xmlns:gml="http://www.opengis.net/gml" xmlns:gmd="http://www.isotc211.org/2005/gmd">
    <csw:Query typeNames="gmd:MD_Metadata">
        <csw:ElementSetName>brief</csw:ElementSetName>
        <csw:Constraint version="1.1.0">
            <ogc:Filter>
                <ogc:Or>
                    <ogc:And>
                        <ogc:PropertyIsLike wildCard="*" singleChar="?" escapeChar="\\" matchCase="false">
                            <ogc:PropertyName>dc:{field_to_query}</ogc:PropertyName>
                            <ogc:Literal>temperature</ogc:Literal>
                        </ogc:PropertyIsLike>
                        <ogc:BBOX>
                            <ogc:PropertyName>apiso:BoundingBox</ogc:PropertyName>
                                <gml:Envelope>
                                    <gml:lowerCorner>{lowerCorner}</gml:lowerCorner>
                                    <gml:upperCorner>{upperCorner}</gml:upperCorner>
                                </gml:Envelope>
                        </ogc:BBOX>
                    </ogc:And>
                    <ogc:And>
                        <ogc:PropertyIsLike wildCard="*" singleChar="?" escapeChar="\\" matchCase="false">
                            <ogc:PropertyName>dc:{field_to_query}</ogc:PropertyName>
                            <ogc:Literal>area</ogc:Literal>
                        </ogc:PropertyIsLike>
                        <ogc:BBOX>
                            <ogc:PropertyName>apiso:BoundingBox</ogc:PropertyName>
                                <gml:Envelope>
                                    <gml:lowerCorner>{lowerCorner}</gml:lowerCorner>
                                    <gml:upperCorner>{upperCorner}</gml:upperCorner>
                                </gml:Envelope>
                        </ogc:BBOX>
                    </ogc:And>
                </ogc:Or>
            </ogc:Filter>
        </csw:Constraint>
    </csw:Query>
</csw:GetRecords>

In [56]:
!curl -s -H "Content-Type: text/xml" -X POST -d @test_xml_query_anytext_on_field_AND_BBOX_OR_testing_3.xml {csw_test_endpoint} | grep -oP 'numberOfRecordsMatched="\K[^"]+'

4


I get 4 records (as expected it could have been 3 or 4)

#### Python

In [57]:
# Create filters

text_to_query_1 = 'temperature'
text_to_query_2 = 'area'


text_filter1 = PropertyIsLike(
    propertyname=f"dc:{field_to_query}",
    literal=f"{text_to_query_1}",
    wildCard="*",
    singleChar="?",
    escapeChar="\\",
)
text_filter2 = PropertyIsLike(
    propertyname=f"dc:{field_to_query}",
    literal=f"{text_to_query_2}",
    wildCard="*",
    singleChar="?",
    escapeChar="\\",
)
bbox_filter = BBox(lowerCorner.split() + upperCorner.split())

# Combine filters
filter1 = And([text_filter1, bbox_filter])
filter2 = And([text_filter2, bbox_filter])
filter = Or([filter1, filter2])

# Use the filter in a CSW getrecords request
csw.getrecords2(
    constraints=[filter],
    esn="brief",
    startposition=1,
    maxrecords=5,
    outputschema="http://www.isotc211.org/2005/gmd",
)

In [58]:
csw.results['matches']

4

# Note



## PropertyIsEqualTo vs PropertyIsEqualTo

* Anytext can be used with `PropertyIsLike` or `PropertyIsEqualTo`
  when using `PropertyIsEqualTo` it requires `PropertyName` to be set:

  ```xml
              <ogc:PropertyIsEqualTo>
               <ogc:PropertyName>dc:{field_to_query}</ogc:PropertyName>
               <ogc:Literal>{text_to_quey}</ogc:Literal>
            </ogc:PropertyIsEqualTo>
  ```

#### CSW XML

In [60]:
%%writetemplate test_xml_query_anytext_ofield_equal_to.xml
<?xml version="1.0" encoding="UTF-8"?>
<csw:GetRecords xmlns:csw="http://www.opengis.net/cat/csw/2.0.2"
               xmlns:ogc="http://www.opengis.net/ogc"
               xmlns:gml="http://www.opengis.net/gml"
               xmlns:ogc_filter="urn:x-ogc:def:filter:OGC:1.0:equals"
               service="CSW" version="2.0.2"
               resultType="results" startPosition="1" maxRecords="10">
   <csw:Query typeNames="csw:Record">
      <csw:ElementSetName>{ElementSetName}</csw:ElementSetName>
      <csw:Constraint version="1.1.0">
         <ogc:Filter>
            <ogc:PropertyIsEqualTo>
               <ogc:PropertyName>dc:{field_to_query}</ogc:PropertyName>
               <ogc:Literal>{text_to_query}</ogc:Literal>
            </ogc:PropertyIsEqualTo>
         </ogc:Filter>
      </csw:Constraint>
   </csw:Query>
</csw:GetRecords>

In [61]:
!curl -s -H "Content-Type: text/xml" -X POST -d @test_xml_query_anytext_ofield_equal_to.xml {csw_test_endpoint} | grep -oP 'numberOfRecordsMatched="\K[^"]+'

58611


#### Python

In [62]:
# Create filter
filter = PropertyIsEqualTo(
    propertyname=f"dc:{field_to_query}", literal=f"{text_to_query}"
)

# Use the filter in a CSW getrecords request
csw.getrecords2(
    constraints=[filter], esn=f"{ElementSetName}", startposition=1, maxrecords=10
)

In [63]:
csw.results['matches']

58611

## Retrieve data from Python Query

In [64]:
# Create filter
filter = PropertyIsEqualTo(
    propertyname=f"dc:{field_to_query}", literal=f"{text_to_query}"
)

# Use the filter in a CSW getrecords request
csw.getrecords2(
    constraints=[filter], esn=f"{ElementSetName}", startposition=1, maxrecords=10
)

In [71]:
print("Found {} records.\n".format(len(csw.records.keys())))
for key, value in list(csw.records.items()):
    # print(key)

    # display(HTML("<pre>{}</pre>".format(prettify_xml(value.xml))))
    print("Title: [{}]\nID: {}\n".format(value.title, key))
    filtered_attributes = [attr for attr in dir(value) if not attr.startswith("_")]
    print('Attributes: ', '\n', filtered_attributes, '\n')
    msg = "geolink: {geolink}\nscheme: {scheme}\nURL: {url}\n".format
    for ref in value.references:
        print(msg(geolink=sniff_link(ref["url"]), **ref))
    print("#########################################################", "\n")

Found 10 records.

Title: [GPS Profiles Fimbulisen Ice Shelf 2009-2012]
ID: 664d3c4c-d660-4fc6-b8ee-ee0443c0166c

Attributes:  
 ['abstract', 'accessrights', 'alternative', 'bbox', 'bbox_wgs84', 'contributor', 'coverage', 'created', 'creator', 'date', 'format', 'identifier', 'identifiers', 'ispartof', 'issued', 'language', 'license', 'modified', 'publisher', 'rdf', 'references', 'relation', 'rights', 'rightsholder', 'source', 'spatial', 'subjects', 'temporal', 'title', 'type', 'uris', 'xml'] 

geolink: WWW:DOWNLOAD
scheme: WWW:DOWNLOAD-1.0-http--download
URL: https://api.npolar.no/dataset/664d3c4c-d660-4fc6-b8ee-ee0443c0166c/_file/_all/?filename=npolar.2016.664d3c4c-data&format=zip

######################################################### 

Title: [TW-ICE2017: Environmental and phytoplankton data from the glacier front and Kongsfjorden cruises in 2017]
ID: 56c2cd62-665b-4228-ab5d-d0122748d94c

Attributes:  
 ['abstract', 'accessrights', 'alternative', 'bbox', 'bbox_wgs84', 'contributo